In [1]:
import pandas as pd
import numpy as np
from igraph import Graph

In [2]:
# Function to obatain fastgreedy communities
def get_fastgreedy_communities(net):
    g = Graph.Load(net)
    dendrogram = g.community_fastgreedy(weights="weight")
    clusters = dendrogram.as_clustering()
    return g, clusters

In [3]:
# Build dataframe communities
def format_communities(g, clusters):
    
    def check_seller_role(row):
        if '_seller' in row['club_name']:
            row['club_name'] = row['club_name'].replace('_seller', '')
            row['role'] = 'seller'
        else:
            row['role'] = 'buyer'
        return row
    
    df_clusters = pd.DataFrame()
    for i, cluster in enumerate(clusters):
        cluster_idx = [i for c in cluster]
        club_name = [g.vs[c]["name"] for c in cluster]
        df_aux = pd.DataFrame({'cluster_idx': cluster_idx, 'club_name': club_name})
        df_clusters = pd.concat([df_clusters, df_aux])
    
    # Add league information
    df_league = pd.read_csv('resources/club_league_v2.csv')
    df_league_sellers = df_league.copy()
    df_league_sellers['club_name'] = df_league_sellers['club_name'] + '_seller'
    df_clusters = pd.merge(df_clusters, df_league, how='left', on='club_name')
    df_clusters = pd.merge(df_clusters, df_league_sellers, how='left', on='club_name')
    df_clusters['league_name'] = df_clusters['league_name_x'].fillna(df_clusters['league_name_y'])
    df_clusters.drop(columns=['league_name_x', 'league_name_y'], inplace=True)
    
    df_clusters = df_clusters.apply(check_seller_role, axis=1)
    return df_clusters

In [4]:
# Get cluster's league composition
def get_league_composition(df_clusters):
    df_clusters_stats = df_clusters.groupby(['cluster_idx', 'role','league_name']).club_name.count().reset_index()
    df_clusters_stats_aux = df_clusters_stats.groupby(['cluster_idx', 'role']).club_name.sum().reset_index()
    df_clusters_stats = pd.merge(df_clusters_stats, df_clusters_stats_aux, how='inner', on=['cluster_idx', 'role'])
    df_clusters_stats.rename(columns={'club_name_x': 'count_league'}, inplace=True)
    df_clusters_stats['perc'] = 100*df_clusters_stats['count_league']/df_clusters_stats['club_name_y']
    df_clusters_stats.drop(columns='club_name_y', inplace=True)
    return df_clusters_stats

In [5]:
def filter_small_clusters(df_cluster_stats, min_clubs):
    df_cluster_stats_aux = df_cluster_stats[df_cluster_stats['role'] == 'buyer'].groupby('cluster_idx').count_league.sum().reset_index()
    df_cluster_stats_aux = df_cluster_stats_aux[df_cluster_stats_aux['count_league'] >= min_clubs]
    df_cluster_stats = pd.merge(df_cluster_stats, df_cluster_stats_aux[['cluster_idx']], how='inner', on='cluster_idx')
    return df_cluster_stats

In [6]:
# Do all function
def get_communities(net):
    g, clusters = get_fastgreedy_communities(net)
    df_clusters = format_communities(g, clusters)
    df_cluster_stats = get_league_composition(df_clusters)
    df_cluster_stats = filter_small_clusters(df_cluster_stats, 8)
    return df_clusters, df_cluster_stats

In [28]:
# Given a net, returns a dataframe with the centrality of each team 
# in the clusters found by the fastgreedy algorithm
def get_centralities_clusters(net):
    
    df_clusters, df_cluster_stats = get_communities(net)
    g = Graph.Load(net)
    ids = np.array(g.vs['id'])
    strengths = np.array(g.strength(weights='weight'))
    
    df_clusters_indexed = df_clusters.set_index('cluster_idx')
    cluster_list = df_cluster_stats['cluster_idx'].unique().tolist()

    d = {'club_name':[], 'league': [], 'cluster':[], 'role':[], 'centrality':[]}

    for idx in df_clusters_indexed.index.unique():

        if idx not in cluster_list:
            continue

        cluster = df_clusters_indexed.loc[idx]
        league = cluster[cluster['role']=='buyer'].league_name.mode() # Most common league

        teams = cluster['club_name'].values
        strengths_cluster = strengths[np.where(np.isin(ids, teams))]

        for i, t in enumerate(teams):

            if cluster.iloc[i]['role'] == 'seller':
                club_strength = strengths[np.where(ids == t + '_seller')[0]]
            else:
                club_strength = strengths[np.where(ids == t)[0]]

            if club_strength.size > 0: 
                centrality = ((club_strength - np.mean(strengths_cluster))/np.std(strengths_cluster))[0]
            else:
                centrality  = 0

            d['club_name'].append(t)
            d['cluster'].append(league.values[0])
            d['league'].append(cluster.iloc[i]['league_name'])
            d['role'].append(cluster.iloc[i]['role'])
            d['centrality'].append(centrality)
            
    return pd.DataFrame(d)    

In [29]:
# To organize the data by clusters and consider only the best teams in terms of centrality 
# (by default the first 3 ones)
def sort_centr_clust(df, role='buyer', number_teams=3):
    df = df[df['role']==role]
    df = df.sort_values(['cluster', 'centrality'], ascending=[True,False])
    df = df.set_index('cluster')
    
    df_reduced = pd.DataFrame(columns=df.columns)
    for i in df.index.unique():
        df_reduced = df_reduced.append(df.loc[i].iloc[:number_teams])
    
    df_reduced = df_reduced.rename(columns={'club_name':'best_{}s'.format(role)})
    df_reduced = df_reduced.drop('role', axis=1)
    df_reduced = df_reduced.set_index([df_reduced.index, np.tile(range(number_teams),df.index.unique().size)+1])  
    df_reduced = df_reduced.rename_axis(['Clusters (most frequent league)', 'Rank'])
    
    return df_reduced

### The most central teams in the communities

Idea: find, for each community, wich are the most central teams 

#### Pre Bosman era (1992-1995)

In [10]:
#CONSIDERING FREQ
df = get_centralities_clusters('nets/1992-1995/freq_net.net')
df_buyer  = sort_centr_clust(df, 'buyer')
df_seller = sort_centr_clust(df, 'seller')
pd.concat([df_buyer, df_seller], axis=1).round(2)

best_buyers  \
Clusters (most frequent league) Rank                             
1 Bundesliga                    1              TSV 1860 Munich   
                                2          1.FC Kaiserslautern   
                                3          Eintracht Frankfurt   
Eredivisie                      1               Vitesse Arnhem   
                                2          Feyenoord Rotterdam   
                                3     Go Ahead Eagles Deventer   
Liga Nos                        1                   SL Benfica   
                                2                  Sporting CP   
                                3               Belenenses SAD   
Ligue 1                         1                 Leeds United   
                                2        FC Girondins Bordeaux   
                                3          Olympique Marseille   
Premier League                  1             Blackburn Rovers   
                                2                Coventry City   
                                3          Sheffield Wednesday   
Premier Liga                    1                Dinamo Moscow   
                                2           Rostselmash Rostov   
                                3             Racing Santander   
Primera Division                1           Real Valladolid CF   
                                2       Deportivo de La Coruña   
                                3                Celta de Vigo   
Serie A                         1                Torino Calcio   
                                2                   SSC Napoli   
                                3            FC Internazionale   

                                                league  centrality  \
Clusters (most frequent league) Rank                                 
1 Bundesliga                    1         1 Bundesliga        1.97   
                                2         1 Bundesliga        1.60   
                                3         1 Bundesliga        1.22   
Eredivisie                      1           Eredivisie        1.69   
                                2           Eredivisie        1.39   
                                3           Eredivisie        1.39   
Liga Nos                        1             Liga Nos        2.58   
                                2             Liga Nos        1.70   
                                3             Liga Nos        1.58   
Ligue 1                         1       Premier League        2.87   
                                2              Ligue 1        1.30   
                                3              Ligue 1        1.30   
Premier League                  1       Premier League        3.53   
                                2       Premier League        2.14   
                                3       Premier League        2.02   
Premier Liga                    1         Premier Liga        2.47   
                                2         Premier Liga        1.87   
                                3     Primera Division        0.98   
Primera Division                1     Primera Division        2.02   
                                2     Primera Division        1.32   
                                3     Primera Division        1.32   
Serie A                         1              Serie A        1.85   
                                2              Serie A        1.61   
                                3              Serie A        1.37   

                                             best_sellers            league  \
Clusters (most frequent league) Rank                                          
1 Bundesliga                    1           Bayern Munich      1 Bundesliga   
                                2          1.FC Nuremberg      1 Bundesliga   
                                3       SG Dynamo Dresden      1 Bundesliga   
Eredivisie                      1           PSV Eindhoven        Eredivisie   
                                2      

**Comments**: if we consider the frequency of the transfers, we can appreciate a very standard situation in which in all the clusters the teams with the highest centrality belong to the most frequent league of that cluster

In [11]:
#CONSIDERING FEE
df = get_centralities_clusters('nets/1992-1995/fee_net.net')
df_buyer  = sort_centr_clust(df, 'buyer')
df_seller = sort_centr_clust(df, 'seller')
pd.concat([df_buyer, df_seller], axis=1).round(2)

best_buyers            league  \
Clusters (most frequent league) Rank                                          
1 Bundesliga                    1                Milan AC           Serie A   
                                2           Bayern Munich      1 Bundesliga   
                                3                SS Lazio           Serie A   
Eredivisie                      1            FC Barcelona  Primera Division   
                                2          Ajax Amsterdam        Eredivisie   
                                3           PSV Eindhoven        Eredivisie   
Liga Nos                        1           AC Fiorentina           Serie A   
                                2             Juventus FC           Serie A   
                                3              Arsenal FC    Premier League   
Premier League                  1        Blackburn Rovers    Premier League   
                                2              Chelsea FC    Premier League   
                                3       FC Internazionale           Serie A   
Premier Liga                    1           Real Sociedad  Primera Division   
                                2      CD Logroñés (liq.)  Primera Division   
                                3     UD Salamanca (liq.)  Primera Division   
Primera Division                1                AC Parma           Serie A   
                                2        Middlesbrough FC    Premier League   
                                3             Real Madrid  Primera Division   

                                      centrality         best_sellers  \
Clusters (most frequent league) Rank                                    
1 Bundesliga                    1           3.25          Juventus FC   
                                2           2.14  Olympique Marseille   
                                3           1.74        Torino Calcio   
Eredivisie                      1           4.66        PSV Eindhoven   
                                2           0.76       RSC Anderlecht   
                                3           0.63        Real Sociedad   
Liga Nos                        1           2.45        Bayern Munich   
                                2           2.18         UC Sampdoria   
                                3           1.03           SL Benfica   
Premier League                  1           4.27             Red Star   
                                2           3.26       Crystal Palace   
                                3           3.14         Norwich City   
Premier Liga                    1           2.65         FC U Craiova   
                                2          -0.47  UD Salamanca (liq.)   
                                3          -0.47             Española   
Primera Division                1           2.43         FC Barcelona   
                                2           2.30    Tottenham Hotspur   
                                3           2.25            São Paulo   

                                                league  centrality  
Clusters (most frequent league) Rank                                
1 Bundesliga                    1              Serie A        1.80  
                                2              Ligue 1        1.79  
                                3              Serie A        0.44  
Eredivisie                      1           Eredivisie        1.70  
                                2              Belgium        1.08  
                                3     Primera Division        0.77  
Liga Nos                        1         1 Bundesliga        3.11  
                                2              Serie A        1.52  
                                3             Liga Nos        1.28  
Premier League                  1               Serbia        1.92  
                                2       Premier League        1.11  
                                3       Premier League        1.07  
Premier Liga                    1     

**Comments**: if we consider the money involved in the transfers (FEE), we can appreciate already a more heterogenous situation. In particular, we can see that in many cases the teams that invest (or gain) the highest amount of money in a cluster don't belong to the dominant league of that cluster. For example, the teams of Serie A were clearly doing the higest investments in other leagues. 

From the perspective of the buyers:
- Lig Nos is completely dominated by Serie A and premier league teams
- Russian league is completely dominated by spanish teams
- Premier League also in this case is much more compact than the other leagues

### Early era 1996-1999

In [12]:
#CONSIDERING FREQ
df = get_centralities_clusters('nets/1996-1999/freq_net.net')
df_buyer  = sort_centr_clust(df, 'buyer')
df_seller = sort_centr_clust(df, 'seller')
pd.concat([df_buyer, df_seller], axis=1).round(2)

best_buyers  \
Clusters (most frequent league) Rank                           
1 Bundesliga                    1        Bayer 04 Leverkusen   
                                2                 Hertha BSC   
                                3              Bayern Munich   
Eredivisie                      1             Ajax Amsterdam   
                                2              PSV Eindhoven   
                                3               RKC Waalwijk   
Liga Nos                        1                 SL Benfica   
                                2                   FC Porto   
                                3                Sporting CP   
Ligue 1                         1        Olympique Marseille   
                                2      FC Girondins Bordeaux   
                                3        Paris Saint-Germain   
Premier League                  1           Blackburn Rovers   
                                2               Derby County   
                                3               Leeds United   
Premier Liga                    1             Torpedo Moscow   
                                2             Spartak Moscow   
                                3                CSKA Moscow   
Primera Division                1     Deportivo de La Coruña   
                                2                Valencia CF   
                                3           Racing Santander   
Serie A                         1            Bologna FC 1909   
                                2          FC Internazionale   
                                3                 AC Perugia   

                                                league  centrality  \
Clusters (most frequent league) Rank                                 
1 Bundesliga                    1         1 Bundesliga        1.51   
                                2         1 Bundesliga        1.29   
                                3         1 Bundesliga        1.07   
Eredivisie                      1           Eredivisie        1.94   
                                2           Eredivisie        1.94   
                                3           Eredivisie        1.16   
Liga Nos                        1             Liga Nos        2.87   
                                2             Liga Nos        1.70   
                                3             Liga Nos        1.57   
Ligue 1                         1              Ligue 1        2.75   
                                2              Ligue 1        1.59   
                                3              Ligue 1        1.59   
Premier League                  1       Premier League        2.27   
                                2       Premier League        2.27   
                                3       Premier League        2.27   
Premier Liga                    1         Premier Liga        1.99   
                                2         Premier Liga        1.26   
                                3         Premier Liga        1.26   
Primera Division                1     Primera Division        2.03   
                                2     Primera Division        2.03   
                                3     Primera Division        1.51   
Serie A                         1              Serie A        1.87   
                                2              Serie A        1.78   
                                3              Serie A        1.69   

                                                  best_sellers  \
Clusters (most frequent league) Rank                             
1 Bundesliga                    1          Bayer 04 Leverkusen   
                                2                Bayern Munich   
                                3     Borussia Mönchengladbach   
Eredivisie                      1               Vitesse Arnhem   
                                2               RSC Anderlecht   
                                3          Feyenoord Rotterdam   
Liga Nos                        1  

**Comments**: still standard situation 

In [15]:
#CONSIDERING FEE
df = get_centralities_clusters('nets/1996-1999/fee_net.net')
df_buyer  = sort_centr_clust(df, 'buyer')
df_seller = sort_centr_clust(df, 'seller')
pd.concat([df_buyer, df_seller], axis=1).round(2)

best_buyers  \
Clusters (most frequent league) Rank                           
1 Bundesliga                    1             Ajax Amsterdam   
                                2              Bayern Munich   
                                3        Bayer 04 Leverkusen   
Liga Nos                        1     Deportivo de La Coruña   
                                2           Newcastle United   
                                3           Middlesbrough FC   
Ligue 1                         1          FC Internazionale   
                                2                  AS Monaco   
                                3              PSV Eindhoven   
Premier League                  1               Leeds United   
                                2           Blackburn Rovers   
                                3               Derby County   
Premier Liga                    1                Real Madrid   
                                2                   SS Lazio   
                                3                   Milan AC   
Serie A                         1               FC Barcelona   
                                2                Juventus FC   
                                3          Manchester United   

                                                league  centrality  \
Clusters (most frequent league) Rank                                 
1 Bundesliga                    1           Eredivisie        1.61   
                                2         1 Bundesliga        1.60   
                                3         1 Bundesliga        1.41   
Liga Nos                        1     Primera Division        4.08   
                                2       Premier League        3.11   
                                3       Premier League        1.65   
Ligue 1                         1              Serie A        4.22   
                                2              Ligue 1        1.90   
                                3           Eredivisie        1.51   
Premier League                  1       Premier League        3.39   
                                2       Premier League        3.18   
                                3       Premier League        2.77   
Premier Liga                    1     Primera Division        4.08   
                                2              Serie A        2.42   
                                3              Serie A        1.97   
Serie A                         1     Primera Division        2.99   
                                2              Serie A        2.00   
                                3       Premier League        1.15   

                                                 best_sellers  \
Clusters (most frequent league) Rank                            
1 Bundesliga                    1              Vitesse Arnhem   
                                2                     AS Roma   
                                3               VfB Stuttgart   
Liga Nos                        1                   Palmeiras   
                                2            Blackburn Rovers   
                                3                 Juventus FC   
Ligue 1                         1                FC Barcelona   
                                2                    SS Lazio   
                                3                 Real Madrid   
Premier League                  1            Newcastle United   
                                2     Wolverhampton Wanderers   
                                3             Crewe Alexandra   
Premier Liga                    1                  Arsenal FC   
                                2                    AC Parma   
                                3                UC Sampdoria   
Serie A                         1                    Milan AC   
                                2              Ajax Amsterdam   
                                3               PSV Eindhoven   

                                                league  centrality

**Comments**: again, if we consider the money involved in the transfers (FEE), the situation is much more heterogienic and the market is changing much more, with bigs teams that are always moving to different markets. Incredible the stability of Premier League

### Mature era 2000 - 2010

In [16]:
#CONSIDERING FREQ
df = get_centralities_clusters('nets/2000-2010/freq_net.net')
df_buyer  = sort_centr_clust(df, 'buyer')
df_seller = sort_centr_clust(df, 'seller')
pd.concat([df_buyer, df_seller], axis=1).round(2)

best_buyers            league  \
Clusters (most frequent league) Rank                                          
1 Bundesliga                    1           VfL Wolfsburg      1 Bundesliga   
                                2            Hamburger SV      1 Bundesliga   
                                3        SV Werder Bremen      1 Bundesliga   
Liga Nos                        1                FC Porto          Liga Nos   
                                2              SL Benfica          Liga Nos   
                                3                SC Braga          Liga Nos   
Ligue 1                         1     Paris Saint-Germain           Ligue 1   
                                2     Olympique Marseille           Ligue 1   
                                3          Olympique Lyon           Ligue 1   
Premier League                  1       Tottenham Hotspur    Premier League   
                                2               Fulham FC    Premier League   
                                3            Liverpool FC    Premier League   
Premier Liga                    1             Rubin Kazan      Premier Liga   
                                2        Saturn Ramenskoe      Premier Liga   
                                3        Lokomotiv Moscow      Premier Liga   
Primera Division                1         Atlético Madrid  Primera Division   
                                2             Real Madrid  Primera Division   
                                3              Chelsea FC    Premier League   
Serie A                         1       FC Internazionale           Serie A   
                                2          Udinese Calcio           Serie A   
                                3             Juventus FC           Serie A   

                                      centrality         best_sellers  \
Clusters (most frequent league) Rank                                    
1 Bundesliga                    1           2.88  Feyenoord Rotterdam   
                                2           1.97       Ajax Amsterdam   
                                3           1.81        PSV Eindhoven   
Liga Nos                        1           3.62             FC Porto   
                                2           2.54          CD Nacional   
                                3           1.40           SL Benfica   
Ligue 1                         1           2.34  Olympique Marseille   
                                2           2.09            AS Monaco   
                                3           1.59       Olympique Lyon   
Premier League                  1           1.76         Liverpool FC   
                                2           1.71    Tottenham Hotspur   
                                3           1.56    Manchester United   
Premier Liga                    1           2.43       Spartak Moscow   
                                2           2.02          CSKA Moscow   
                                3           1.81       Torpedo Moscow   
Primera Division                1           1.71          River Plate   
                                2           1.65          Real Madrid   
                                3           1.48      Atlético Madrid   
Serie A                         1           2.33    FC Internazionale   
                                2           1.80          Juventus FC   
                                3           1.66             AC Parma   

                                                league  centrality  
Clusters (most frequent league) Rank                                
1 Bundesliga                    1           Eredivisie        2.27  
                                2           Eredivisie        1.51  
                                3           Eredivisie        1.28  
Liga Nos                        1             Liga Nos        2.90  
                                2             Liga Nos        1.45  
                                3             Liga Nos        1.3

**Comments**: still static

In [18]:
#CONSIDERING FEE
df = get_centralities_clusters('nets/2000-2010/fee_net.net')
df_buyer  = sort_centr_clust(df, 'buyer')
df_seller = sort_centr_clust(df, 'seller')
pd.concat([df_buyer, df_seller], axis=1).round(2)

best_buyers            league  \
Clusters (most frequent league) Rank                                          
1 Bundesliga                    1              SL Benfica          Liga Nos   
                                2            Hamburger SV      1 Bundesliga   
                                3          Ajax Amsterdam        Eredivisie   
Liga Nos                        1             Real Madrid  Primera Division   
                                2              Chelsea FC    Premier League   
                                3               Fulham FC    Premier League   
Ligue 1                         1          Olympique Lyon           Ligue 1   
                                2     Olympique Marseille           Ligue 1   
                                3               AS Monaco           Ligue 1   
Premier League                  1            Liverpool FC    Premier League   
                                2       Tottenham Hotspur    Premier League   
                                3       Manchester United    Premier League   
Primera Division                1         Manchester City    Premier League   
                                2            FC Barcelona  Primera Division   
                                3                SS Lazio           Serie A   
Serie A                         1       FC Internazionale           Serie A   
                                2             Juventus FC           Serie A   
                                3                 AS Roma           Serie A   

                                      centrality       best_sellers  \
Clusters (most frequent league) Rank                                  
1 Bundesliga                    1           3.07      Bayern Munich   
                                2           2.75          São Paulo   
                                3           2.27     RSC Anderlecht   
Liga Nos                        1           3.31        Juventus FC   
                                2           2.37           FC Porto   
                                3           0.21       Liverpool FC   
Ligue 1                         1           5.01            RC Lens   
                                2           3.07   Stade Rennais FC   
                                3           1.59         LOSC Lille   
Premier League                  1           3.80  Tottenham Hotspur   
                                2           3.60     Ajax Amsterdam   
                                3           3.46   Newcastle United   
Primera Division                1           3.55  FC Internazionale   
                                2           3.50        Valencia CF   
                                3           1.83         Arsenal FC   
Serie A                         1           2.82           AC Parma   
                                2           2.70        Real Madrid   
                                3           1.32           SS Lazio   

                                                league  centrality  
Clusters (most frequent league) Rank                                
1 Bundesliga                    1         1 Bundesliga        1.98  
                                2               Brazil        1.60  
                                3              Belgium        1.38  
Liga Nos                        1              Serie A        0.98  
                                2             Liga Nos        0.85  
                                3       Premier League        0.80  
Ligue 1                         1              Ligue 1        1.74  
                                2              Ligue 1        1.62  
                                3              Ligue 1        1.15  
Premier League                  1       Premier League        1.29  
                                2           Eredivisie        1.27  
                                3       Premier League        1.24  
Primera Division                1              Serie A        2.70  
         

**Comments**: more stable situation in Italy and France (but notice that we have taken in consideration a longer time, from 2000 to 2010)

From the perspective of the buyers:
- Lig Nos is completely dominated by spanish and english teams (Real Madrid and Chelsea)
- Big premier leagues teamsn invest out of england (Manch city in spain), but big european teams invest in england
- Big italian teams now invest in italy

### Contemporary era 2010 - 2015

In [26]:
#CONSIDERING FREQ
df = get_centralities_clusters('nets/2010-2015/freq_net.net')
df_buyer  = sort_centr_clust(df, 'buyer')
df_seller = sort_centr_clust(df, 'seller')
pd.concat([df_buyer, df_seller], axis=1).round(2)

best_buyers            league  \
Clusters (most frequent league) Rank                                          
1 Bundesliga                    1          1.FSV Mainz 05      1 Bundesliga   
                                2           VfL Wolfsburg      1 Bundesliga   
                                3            Hamburger SV      1 Bundesliga   
Eredivisie                      1           PSV Eindhoven        Eredivisie   
                                2            FC Groningen        Eredivisie   
                                3          Ajax Amsterdam        Eredivisie   
Liga Nos                        1              SL Benfica          Liga Nos   
                                2              Chelsea FC    Premier League   
                                3              Sevilla FC  Primera Division   
Ligue 1                         1        Stade Rennais FC           Ligue 1   
                                2              Granada CF  Primera Division   
                                3     Olympique Marseille           Ligue 1   
Premier League                  1     Queens Park Rangers    Premier League   
                                2            Liverpool FC    Premier League   
                                3            Norwich City    Premier League   
Premier Liga                    1             Rubin Kazan      Premier Liga   
                                2          Spartak Moscow      Premier Liga   
                                3            FK Krasnodar      Premier Liga   
Serie A                         1               Genoa CFC           Serie A   
                                2                Parma FC           Serie A   
                                3             Juventus FC           Serie A   

                                      centrality         best_sellers  \
Clusters (most frequent league) Rank                                    
1 Bundesliga                    1           1.62           Chelsea FC   
                                2           1.36        VfL Wolfsburg   
                                3           1.36  Bayer 04 Leverkusen   
Eredivisie                      1           2.16  Feyenoord Rotterdam   
                                2           1.71           AZ Alkmaar   
                                3           1.42       Ajax Amsterdam   
Liga Nos                        1           1.81           SL Benfica   
                                2           1.62         FC Barcelona   
                                3           1.56      Atlético Madrid   
Ligue 1                         1           2.03            AS Monaco   
                                2           1.66     Stade Rennais FC   
                                3           1.39       Olympique Lyon   
Premier League                  1           2.20    Manchester United   
                                2           1.85    Tottenham Hotspur   
                                3           1.77      Manchester City   
Premier Liga                    1           1.97       Spartak Moscow   
                                2           1.82    Anzhi Makhachkala   
                                3           1.68      Kuban Krasnodar   
Serie A                         1           2.42       Udinese Calcio   
                                2           2.34    FC Internazionale   
                                3           1.81              AS Roma   

                                                league  centrality  
Clusters (most frequent league) Rank                                
1 Bundesliga                    1       Premier League        2.23  
                                2         1 Bundesliga        2.05  
                                3         1 Bundesliga        1.27  
Eredivisie                      1           Eredivisie        2.91  
                                2           Eredivisie        2.31  
                                3           Eredivisie        1.8

**Comments**: Something start to change, less consistency of the clusters

In [27]:
#CONSIDERING FEE
df = get_centralities_clusters('nets/2010-2015/fee_net.net')
df_buyer  = sort_centr_clust(df, 'buyer')
df_seller = sort_centr_clust(df, 'seller')
pd.concat([df_buyer, df_seller], axis=1).round(2)

best_buyers            league  \
Clusters (most frequent league) Rank                                          
1 Bundesliga                    1           Bayern Munich      1 Bundesliga   
                                2           VfL Wolfsburg      1 Bundesliga   
                                3                AC Milan           Serie A   
Eredivisie                      1            Swansea City    Premier League   
                                2          Udinese Calcio           Serie A   
                                3           PSV Eindhoven        Eredivisie   
Ligue 1                         1        Newcastle United    Premier League   
                                2                FC Porto          Liga Nos   
                                3     Queens Park Rangers    Premier League   
Premier League                  1          Southampton FC    Premier League   
                                2       Manchester United    Premier League   
                                3         Manchester City    Premier League   
Primera Division                1             Real Madrid  Primera Division   
                                2         Atlético Madrid  Primera Division   
                                3       FC Internazionale           Serie A   
Serie A                         1     Paris Saint-Germain           Ligue 1   
                                2             Juventus FC           Serie A   
                                3                 AS Roma           Serie A   

                                      centrality         best_sellers  \
Clusters (most frequent league) Rank                                    
1 Bundesliga                    1           1.48              AS Roma   
                                2           1.28      Manchester City   
                                3           1.17          Juventus FC   
Eredivisie                      1           2.75           AZ Alkmaar   
                                2           2.74           Olympiacos   
                                3           1.97        SC Heerenveen   
Ligue 1                         1           3.34    Anzhi Makhachkala   
                                2           2.87  Olympique Marseille   
                                3           2.40     AS Saint-Étienne   
Premier League                  1           3.55           Chelsea FC   
                                2           2.82      Atlético Madrid   
                                3           2.70          Real Madrid   
Primera Division                1           3.38             FC Porto   
                                2           2.25           SL Benfica   
                                3           1.54    Tottenham Hotspur   
Serie A                         1           3.12       Udinese Calcio   
                                2           2.43           US Palermo   
                                3           1.59    Manchester United   

                                                league  centrality  
Clusters (most frequent league) Rank                                
1 Bundesliga                    1              Serie A        1.46  
                                2       Premier League        1.02  
                                3              Serie A        0.93  
Eredivisie                      1           Eredivisie        1.82  
                                2               Greece        1.44  
                                3           Eredivisie        1.07  
Ligue 1                         1         Premier Liga        3.17  
                                2              Ligue 1        1.48  
                                3              Ligue 1        1.28  
Premier League                  1       Premier League        1.29  
                                2     Primera Division        1.14  
                                3     Primera Division        1.08  
Primera Division                1     

**Comments**: Premier league not anymore compact in the perspective of the sellers

### Nowadays 2015 - 2020

In [24]:
#CONSIDERING FREQ
df = get_centralities_clusters('nets/2015-2020/freq_net.net')
df_buyer  = sort_centr_clust(df, 'buyer')
df_seller = sort_centr_clust(df, 'seller')
pd.concat([df_buyer, df_seller], axis=1).round(2)

best_buyers  \
Clusters (most frequent league) Rank                            
1 Bundesliga                    1                  Sevilla FC   
                                2               VfL Wolfsburg   
                                3                  Everton FC   
Eredivisie                      1      Brighton & Hove Albion   
                                2              Ajax Amsterdam   
                                3               PSV Eindhoven   
Liga Nos                        1                 Sporting CP   
                                2                  SL Benfica   
                                3                    SS Lazio   
Ligue 1                         1              Olympique Lyon   
                                2                    OGC Nice   
                                3                Brentford FC   
Premier League                  1                 Aston Villa   
                                2                   Fulham FC   
                                3            Newcastle United   
Premier Liga                    1        Zenit St. Petersburg   
                                2              Spartak Moscow   
                                3               Dynamo Moscow   
Primera Division                1                   AS Monaco   
                                2     Wolverhampton Wanderers   
                                3               Villarreal CF   
Serie A                         1                     AS Roma   
                                2              ACF Fiorentina   
                                3                   Genoa CFC   

                                                league  centrality  \
Clusters (most frequent league) Rank                                 
1 Bundesliga                    1     Primera Division        3.08   
                                2         1 Bundesliga        1.56   
                                3       Premier League        1.11   
Eredivisie                      1       Premier League        2.98   
                                2           Eredivisie        1.88   
                                3           Eredivisie        1.43   
Liga Nos                        1             Liga Nos        2.74   
                                2             Liga Nos        2.14   
                                3              Serie A        1.88   
Ligue 1                         1              Ligue 1        2.19   
                                2              Ligue 1        1.38   
                                3       Premier League        1.03   
Premier League                  1       Premier League        2.10   
                                2       Premier League        1.93   
                                3       Premier League        1.24   
Premier Liga                    1         Premier Liga        2.35   
                                2         Premier Liga        2.05   
                                3         Premier Liga        1.32   
Primera Division                1              Ligue 1        2.72   
                                2       Premier League        1.45   
                                3     Primera Division        1.37   
Serie A                         1              Serie A        2.09   
                                2              Serie A        1.84   
                                3              Serie A        1.42   

                                             best_sellers            league  \
Clusters (most frequent league) Rank                                          
1 Bundesliga                    1            FC Barcelona  Primera Division   
                                2       Borussia Dortmund      1 Bundesliga   
                                3             Real Madrid  Primera Division   
Eredivisie                      1           PSV Eindhoven        Eredivisie   
                                2          Ajax Amsterdam       

**Comments**: In the last 5 yeras the market seems to be more open also in terms of frequency: Bundesliga, Eredivise and Primiera Division are dominated bybuyers of other leagues, and big teams (as Barcelona, Chelsea, PSG, Real Madrid) are big sellers in other leagues

In [25]:
#CONSIDERING FREQ
df = get_centralities_clusters('nets/2015-2020/fee_net.net')
df_buyer  = sort_centr_clust(df, 'buyer')
df_seller = sort_centr_clust(df, 'seller')
pd.concat([df_buyer, df_seller], axis=1).round(2)

best_buyers            league  \
Clusters (most frequent league) Rank                                           
1 Bundesliga                    1               RB Leipzig      1 Bundesliga   
                                2              Juventus FC           Serie A   
                                3               Chelsea FC    Premier League   
Liga Nos                        1               LOSC Lille           Ligue 1   
                                2           Ajax Amsterdam        Eredivisie   
                                3                 SS Lazio           Serie A   
Premier League                  1                AS Monaco           Ligue 1   
                                2               SSC Napoli           Serie A   
                                3          Manchester City    Premier League   
Premier Liga                    1      Bayer 04 Leverkusen      1 Bundesliga   
                                2     Zenit St. Petersburg      Premier Liga   
                                3             Leeds United    Premier League   
Primera Division                1        Manchester United    Premier League   
                                2          Atlético Madrid  Primera Division   
                                3              Real Madrid  Primera Division   
Serie A                         1      Paris Saint-Germain           Ligue 1   
                                2        FC Internazionale           Serie A   
                                3               Everton FC    Premier League   

                                      centrality     best_sellers  \
Clusters (most frequent league) Rank                                
1 Bundesliga                    1           3.31      RB Salzburg   
                                2           2.93      Real Madrid   
                                3           2.67    Dinamo Zagreb   
Liga Nos                        1           2.73         SC Braga   
                                2           2.49        São Paulo   
                                3           2.45         KAA Gent   
Premier League                  1           4.62       LOSC Lille   
                                2           3.75          AS Roma   
                                3           3.50  Atlético Madrid   
Premier Liga                    1           3.88    Real Sociedad   
                                2           3.29      River Plate   
                                3           2.05      SC Freiburg   
Primera Division                1           2.24      Juventus FC   
                                2           1.85       SL Benfica   
                                3           1.33       Chelsea FC   
Serie A                         1           1.96        AS Monaco   
                                2           1.76     FC Barcelona   
                                3           1.29  Manchester City   

                                                league  centrality  
Clusters (most frequent league) Rank                                
1 Bundesliga                    1              Austria        2.73  
                                2     Primera Division        1.09  
                                3              Croatia        0.84  
Liga Nos                        1             Liga Nos        2.30  
                                2               Brazil        1.02  
                                3              Belgium        1.01  
Premier League                  1              Ligue 1        2.88  
                                2              Serie A        2.56  
                                3     Primera Division        1.78  
Premier Liga                    1     Primera Division        1.52  
                                2            Argentina        1.50  
                                3         1 Bundesliga        1.36  
Primera Division                1              Serie A        1.74  
                              

**Comments**: the situation is completely open in terms of quantity of money spended: Franch teams make the most important investement outside (no more french cluster), and the only league with two teams belonging to that league is the spanish one (Ateltico and Real). Even in Premier League the best seller and buyers are not anymore english, and I think this is pretty indicative of how much open is the market nowadays